# Reading french texts

In [5]:
# imports
import pandas as pd
import numpy as np
import json

In [25]:
# fetching data
PATH = "./data/ArticleCompany_2020-11-17/"
coprus = "corpus_check_long_SIREN_UPDATED2"
names = "siren_name_map_clean"

In [26]:
#df_name = pd.read_json(PATH + names +".json")
with open(PATH + names +".json") as json_file: 
    dict_names = json.load(json_file) 

with open(PATH + coprus +".json") as json_file: 
    dict_corpus = json.load(json_file) 

## Compter Nombre d'Entreprises sans Articles

In [54]:
dict_count = dict()
#for company in dict_names.keys(): dict_count[company] = 0
for document in dict_corpus:
    #print (document["siren"])
    sir_list = document["siren"][1:-1].split(", ")
    #print (sir_list)
    for siren in sir_list:
        #print (siren)
        if siren in dict_count.keys():
            dict_count[siren] +=1
        else:
            dict_count[siren] = 1
print ("There are",len(dict_count.keys()),"companies with labels out of the", len(dict_names.keys()), "companies")
print ("there are",len(dict_names.keys())-len(dict_count.keys()),"companies with no articles")

There are 28690 companies with labels out of the 30178 companies
there are 1488 companies with no articles


## Quels sont les entreprises sans articles?

In [56]:
dict_no_acticle_companies = dict()
for company in dict_names.keys():
    if company not in dict_count.keys():
        dict_no_acticle_companies[company] = dict_names[company] 

In [58]:
#dict_no_acticle_companies